
environment

In [ ]:
%reload_ext autoreload
%autoreload 2

# univariate

In [ ]:
%script echo skipping...
# noqa


%reload_ext autoreload
%autoreload 2

from tqdm.auto import tqdm
import pandas as pd
import xarray as xr
from scipy.stats import f_oneway

import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.multivariate.manova import MANOVA

from sklearn.pipeline import Pipeline
from src.multimodal.preprocessing import TimeseriesAggregator

preproc_pipe = Pipeline([
    ('time_agg', TimeseriesAggregator(strategy='network')),
])

atlas = 'dosenbach2010'
connectivity_kind = 'tangent'

with xr.open_dataset(f'data/Julia2018/timeseries_{atlas}.nc5') as ds:
    ds.load()
    ds = preproc_pipe.fit_transform(ds)

timeseries = []
y = []
for subject in ds['timeseries'].coords['subject'].values:
    subject_ts = ds['timeseries'].sel(subject=subject).to_pandas()
    subject_ts.insert(0, 'subject', subject)
    y.append((subject, subject[:4]))
    timeseries.append(subject_ts)
timeseries = pd.concat(timeseries).reset_index()
y = pd.DataFrame(y, columns=['subject', 'group']).set_index('subject')['group']

from tsfresh import extract_features
features = extract_features(
    timeseries,
    column_id='subject',
    column_sort='timepoint')

valid_features_mask = (features.std() != 0)
features = features.loc[:, valid_features_mask]

features['y'] = features.index.map(lambda x: x[:4])
features.dropna(axis=1, inplace=True)
features.columns = [
    c.replace('.','_').replace('"','_').replace('(','_').replace(')','_').replace(',','_').replace('-','_').replace(' ','_') for c in features.columns]


In [5]:
# ANOVA
import statsmodels.api as sm

significant_features = []
for feature in tqdm(features.columns):
    if feature == 'y':
        continue
    model = smf.glm(f'y ~ {feature}', features, family=sm.families.Binomial()).fit()
    res = model.summary2()
    pvalue = res.tables[1]['P>|z|'][feature]
    if pvalue < .05:
        # display(res)
        significant_features.append(feature)

print(f'{len(significant_features)}/{len(features.columns) - 1} '  # minus 'y'
      'features are significantly different:', significant_features)


NameError: name 'features' is not defined

In [3]:

feature_formula = ' + '.join(features.columns[:100])
model = smf.glm(f'y ~ {feature_formula}', features, family=sm.families.Binomial()).fit()
model.summary()


/home/morteza/miniforge3/envs/multimodal/lib/python3.12/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/morteza/miniforge3/envs/multimodal/lib/python3.12/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                   Generalized Linear Model Regression Results                    
==================================================================================
Dep. Variable:     ['y[AVGP]', 'y[NVGP]']   No. Observations:                   32
Model:                                GLM   Df Residuals:                        5
Model Family:                    Binomial   Df Model:                           26
Link Function:                      Logit   Scale:                          1.0000
Method:                              IRLS   Log-Likelihood:                -19.083
Date:                    Wed, 17 Apr 2024   Deviance:                       38.167
Time:                            12:44:43   Pearson chi2:                 3.24e+04
No. Iterations:                        26   Pseudo R-squ. (CS):             0.1760
Covariance Type:                nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                      -0.0014   6958.734  -1.97e-07      1.000   -1.36e+04    1.36e+04
CER__sum_values                                                 7.8188   1.73e+06   4.52e-06      1.000   -3.39e+06    3.39e+06
CER__abs_energy                                                 5.7146    6.3e+05   9.08e-06      1.000   -1.23e+06    1.23e+06
CER__mean_abs_change                                           -0.0273   9245.797  -2.95e-06      1.000   -1.81e+04    1.81e+04
CER__mean_change                                                0.2512   4.59e+04   5.47e-06      1.000   -9.01e+04    9.01e+04
CER__mean_second_derivative_central                             0.1138    2.5e+04   4.56e-06      1.000   -4.89e+04    4.89e+04
CER__median                                                     1.6995   3.16e+05   5.37e-06      1.000    -6.2e+05     6.2e+05
CER__mean                                                       0.0631    1.4e+04   4.52e-06      1.000   -2.74e+04    2.74e+04
CER__standard_deviation                                        -0.4904   1.07e+05  -4.57e-06      1.000    -2.1e+05     2.1e+05
CER__variation_coefficient                                  -5.236e-13   3.99e-07  -1.31e-06      1.000   -7.82e-07    7.82e-07
CER__variance                                                   0.0399   5252.106    7.6e-06      1.000   -1.03e+04    1.03e+04
CER__skewness                                                   6.5120   1.47e+06   4.44e-06      1.000   -2.88e+06    2.88e+06
CER__kurtosis                                                   1.2474   2.29e+06   5.44e-07      1.000   -4.49e+06    4.49e+06
CER__root_mean_square                                          -0.4860   1.07e+05  -4.55e-06      1.000   -2.09e+05    2.09e+05
CER__absolute_sum_of_changes                                   -3.3619   1.14e+06  -2.96e-06      1.000   -2.23e+06    2.23e+06
CER__longest_strike_below_mean                                 -0.0552   6.31e+05  -8.76e-08      1.000   -1.24e+06    1.24e+06
CER__longest_strike_above_mean                                  2.8558   8.76e+05   3.26e-06      1.000   -1.72e+06    1.72e+06
CER__count_above_mean                                          -4.7703   4.87e+05  -9.79e-06      1.000   -9.55e+05    9.55e+05
CER__count_below_mean                                          -2.6944   3.29e+05  -8.19e-06      1.000   -6.45e+05    6.45e+05
CER__last_location_of_maximum                                   4.6103   1.49e+06    3.1e-06      1.000   -2.92e+06    2.92e+06
CER__first_location_of_maximum                                  4.6109   1.49e+06    3.1e-06   

In [ ]:

data['connectivity_name'] = data['network_src'] + '_' + data['network_dst']
# data['connectivity'] = data['connectivity'].abs()

# cleanup
data = data.pivot_table(index=['subject', 'group'], columns='connectivity_name', values='connectivity', aggfunc='mean')
# cols = [set(c.split('_')) for c in data.columns
#         if c.split('_')[0] != c.split('_')[1]]
# cc = []
# [cc.append(c) for c in cols if c not in cc]
# cc = [list(c) for c in cc]
# cols = [f'{c[0]}_{c[1]}' for c in cc]

# data = data[cols].reset_index()

# ANOVA
for col in data.columns:
    ols_model = ols(f'{col} ~ C(group)', data=data.reset_index()).fit()
    res = anova_lm(ols_model)
    if res.loc['C(group)', 'PR(>F)'] < .05:
        print(f'[ANOVA] {col} is significant (AVGP != NVGP)')
        display(res)

# MANOVA
manova_model = MANOVA.from_formula('+'.join(data.columns) + '~ C(group)',
                                   data.reset_index())

print('[MANOVA]')
manova_model.mv_test().summary()

In [ ]:
# F oneway anova (using scipy)
import seaborn as sns
import matplotlib.pyplot as plt

data.columns = data.columns.str.replace('_', '\N{left right arrow}')
avgp = data.query('subject.str.contains("AVGP")')
nvgp = data.query('subject.str.contains("NVGP")')


significant_cols = []

for col in data.columns:
    if '\N{left right arrow}' in col:
        f = f_oneway(avgp[col].tolist(), nvgp[col].tolist())
        if f.pvalue < 0.05:
            print(col, f)
            significant_cols.append(col)

In [ ]:
from matplotlib.patches import Rectangle

data_long = data.reset_index().melt(id_vars=['subject', 'group'], var_name='conn', value_name=connectivity_kind)

# \N{downwards arrow}


sns.set_theme('talk', 'ticks', font_scale=2)
g = sns.displot(data=data_long, hue='group',
                x=connectivity_kind,
                col='conn', col_wrap=4,
                element='step',
                facet_kws=dict(sharex=True, sharey=False),
                aspect=1.5, edgecolor='white',
                kind='hist', kde=True, fill=True)

g.set_titles(col_template='{col_name}', fontweight='bold')

for ax in g.axes.flatten():
    if ax.get_title() in significant_cols:
        for s in ax.spines:
            ax.spines[s].set_visible(True)
            ax.spines[s].set_linewidth(8)
            ax.spines[s].set_color('red')
    ax.set_xlabel(None)
    ax.set_ylabel('count')

# move legend of the g to lower right
g._legend.set_bbox_to_anchor([0.83, 0.12])
g._legend.set_title(None)
plt.suptitle(f'Dosenbach2010 {connectivity_kind} (ConnAgg-between)',
             fontweight='bold', fontsize='xx-large')
plt.tight_layout()


# Within Network

the following codes are for future references, they are not valid and do not work.

In [ ]:
%%script echo skipping...

import pandas as pd
from scipy.stats import f_oneway

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.multivariate.manova import MANOVA

from IPython.display import display

data  = pd.read_csv('data/julia2018/derivatives/connectivity/dosenbach2010_{}_cagg-within.csv'.format(connectivity_kind.replace(' ','_')))

data['group'] = data['subject'].apply(lambda x: x[:4])

# cleanup
data = data.pivot_table(index=['subject', 'group'], columns='network', values='connectivity', aggfunc='mean')
# cols = [set(c.split('_')) for c in data.columns
#         if c.split('_')[0] != c.split('_')[1]]
# cc = []
# [cc.append(c) for c in cols if c not in cc]
# cc = [list(c) for c in cc]
# cols = [f'{c[0]}_{c[1]}' for c in cc]

# data = data[cols].reset_index()

# ANOVA
for col in data.columns:
    ols_model = ols(f'{col} ~ C(group)', data=data.reset_index()).fit()
    res = anova_lm(ols_model)
    if res.loc['C(group)', 'PR(>F)'] < .05:
        print(f'[ANOVA] {col} is significant (AVGP != NVGP)')
        display(res)

# MANOVA
manova_model = MANOVA.from_formula('+'.join(data.columns) + '~ C(group)',
                                   data.reset_index())

print('[MANOVA]')
display(manova_model.mv_test().summary())

# F oneway anova (using scipy)
import seaborn as sns
import matplotlib.pyplot as plt

avgp = data.query('subject.str.contains("AVGP")')
nvgp = data.query('subject.str.contains("NVGP")')

significant_cols = []

for col in data.columns:
    f = f_oneway(avgp[col].tolist(), nvgp[col].tolist())
    if f.pvalue < 0.05:
        print(col, f)
        significant_cols.append(col)
    else:
        print(f'{col} is not significant', f)


from matplotlib.patches import Rectangle

data_long = data.reset_index().melt(id_vars=['subject', 'group'], var_name='conn', value_name=connectivity_kind)

# \N{downwards arrow}
sns.set_theme('talk', 'ticks', font_scale=2)
g = sns.displot(data=data_long, hue='group',
                x=connectivity_kind,
                col='conn', col_wrap=4,
                element='step',
                facet_kws=dict(sharex=True, sharey=False),
                aspect=1.5, edgecolor='white',
                kind='hist', kde=True, fill=True)

g.set_titles(col_template='{col_name}', fontweight='bold')

for ax in g.axes.flatten():
    if ax.get_title() in significant_cols:
        for s in ax.spines:
            ax.spines[s].set_visible(True)
            ax.spines[s].set_linewidth(8)
            ax.spines[s].set_color('red')
    ax.set_xlabel(None)
    ax.set_ylabel('count')

# move legend of the g to lower right
g._legend.set_bbox_to_anchor([0.83, 0.12])
g._legend.set_title(None)
plt.suptitle(f'Dosenbach2010 {connectivity_kind} (ConnAgg-within)',
             fontweight='bold', fontsize='xx-large')
plt.tight_layout()
